# Scrape Data from the Nomis API

- Automatically get a list of datasets available via the API and their descriptions
- Get the date each dataset was last updated
- Get the unique parameters for each dataset

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
#| export
import requests

In [ ]:
#| export
def get_nomis_datasets_titles_descriptions():

    """
    Function to get the names and descriptions of all the datasets available via 
    the Nomis api.

    Note:
    Unfortunately the Nomis api requires we specify the parameters (such as 
    geography, age, sex, etc) in order to form a uri and download a dataset.
    These parameters change depending on the data we are dealing with making
    automation very difficult.
    """

    url = "https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json"
    response = requests.get(url)
    data = response.json()
    temp = data['structure']['keyfamilies']['keyfamily']
    nomis_datasets_names = []
    nomis_datasets_descriptions = []
    for dataset in temp:
        nomis_datasets_names.append(dataset['name']['value'])

        if 'description' in dataset:
            nomis_datasets_descriptions.append(dataset['description']['value'])
        else:
            nomis_datasets_descriptions.append(float('nan'))

    url = "https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json"
    response = requests.get(url)
    data = response.json()
    temp = data['structure']['keyfamilies']['keyfamily']
    nomis_datasets_names = []
    nomis_datasets_descriptions = []
    
    for dataset in temp:
        nomis_datasets_names.append(dataset['name']['value'])

        if 'description' in dataset:
            nomis_datasets_descriptions.append(dataset['description']['value'])
        
        else:
            temp_dscptn = ''
            for annotation in dataset['annotations']['annotation']:
                temp_dscptn += str(annotation['annotationtext']) + '\n'
            nomis_datasets_descriptions.append(temp_dscptn)

    return nomis_datasets_names, nomis_datasets_descriptions

In [ ]:
titles, descriptions = get_nomis_datasets_titles_descriptions()
print('Title: ' + titles[0] + '\n' + 'Description: ' + descriptions[0])

Title: Jobseeker's Allowance with rates and proportions
Description: Records the number of people claiming Jobseeker's Allowance (JSA) and National Insurance credits at Jobcentre Plus local offices. This is not an official measure of unemployment, but is the only indicative statistic available for areas smaller than Local Authorities.


In [ ]:
#| export
def get_nomis_last_updated():
    """ 
    Get the date the each dataset was last updated on.
    If no date exists then set value to None.
    """
    last_updated = []
    i = 0
    url = "https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json"
    response = requests.get(url)

    try:
        keyfamilies = response.json()['structure']['keyfamilies']['keyfamily']
        
        for keyfamily in keyfamilies:
            last_updated.append(None) # Ensure there is something in place for each dataset even if there is no last updated value
            annotations = keyfamily['annotations']['annotation']
        
            for annotation in annotations:
                if annotation['annotationtitle'] == 'LastUpdated':
                    last_updated[i] = annotation['annotationtext']
            i += 1
    except:
        return -1

    return last_updated

In [ ]:
# Show the first date in the list
get_nomis_last_updated()[0]

'2023-06-13 07:00:00'

In [ ]:
#| export
def get_nomis_datasets_parameters():

    """
    Function to collect the IDs and parameters for each dataset on the Nomis API.

    How does it work?
    Every dataset has parameter categories e.g. geography, sex, age. Each of these categories
    then has various parameters e.g. age - 0-12, 13-25, etc. To get the parameters we must first
    loop through every dataset and collect the parameter categories and the IDs. Using the IDs
    (unique to each dataset) to specify the dataset and the categories we can then get the 
    available parameters specific to each dataset. These are saved to a dictionary with the keys
    being the parameter categories and the values being the parameters.
    """

    url = "https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json"
    response = requests.get(url)
    data = response.json()

    parameter_names_list = []   # Stores the lists of parameter names for all available datasets
    ids = []                    # Stores the unique id for each dataset
    datasets = data['structure']['keyfamilies']['keyfamily']    # Focuses on the metadata


    """ Loops through available datasets and collects the ID's and available parameter categories e.g. Geography """
    for dataset in datasets:
        dimensions = dataset['components']['dimension']

        parameter_names = {}  # dictionary which stores the parameter categories as the keys and a list of the options for each category for a single dataset
        ids.append(dataset['id'])
        for dimension in dimensions:
            #parameter_names.append(dimension['conceptref'])
            parameter_names[dimension['conceptref']] = None

        parameter_names_list.append(parameter_names)    # stores parameter categories and options for every dataset (list of dictionaries)


    """
    Loops through the ID's which allows us to specify the dataset we want to extract metadata from.
    Then also loops through each parameter category available for each dataset and creates a list
    of all the options for each category. 
    e.g. Geography: ['United Kingdom', 'Great Britain', 'England', 'Wales', 'Scotland', 'Northern Ireland', 'England and Wales'].
    This will then provide a list containing every possible parameter for every dataset

    Took me about 30 mins to run. To avoid running again I saved the list as parameter_names_list.txt
    """
    for i in range(len(ids)):

        for parameter_name in list(parameter_names_list[i].keys()):

            url = 'https://www.nomisweb.co.uk/api/v01/dataset/' + ids[i] + '/' + parameter_name + '.def.sdmx.json'
            response = requests.get(url)
            info = response.json()

            values = info['structure']['codelists']['codelist'][0]['code']
            temp_list = []
            for value in values:
                try:
                    temp_list.append(value['description']['value'])
                except KeyError:
                    continue    # move on if there is no existing value

            parameter_names_list[i][parameter_name] = temp_list

    return parameter_names_list

In [ ]:
params = get_nomis_datasets_parameters()
params[0]

{'GEOGRAPHY': ['United Kingdom',
  'Great Britain',
  'England',
  'Wales',
  'Scotland',
  'Northern Ireland',
  'England and Wales'],
 'SEX': ['Male', 'Female', 'Total'],
 'ITEM': ['Total claimants',
  'Students on vacation',
  'Temporarily stopped',
  'Claimants under 18 years',
  'Married females'],
 'MEASURES': ['claimants', 'workforce', 'active', 'residence'],
 'FREQ': ['Monthly', 'Quarterly', 'Half-yearly, semester', 'Annually']}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()